# Assignment #2: Language models
Author: Pierre Nugues

## Objectives

The objectives of this assignment are to:
* Write a program to find n-gram statistics
* Compute the probability of a sentence
* Know what a language model is
* Write a short report of 1 to 2 pages on the assignment

## Submission

Once you have written all the missing code and run all the cells, you will show it to an instructor that will pass you.

## Organization

* Each group will have to write Python programs to count unigrams, bigrams, and trigrams in a corpus of approximately one million words and to determine the probability of a sentence.
* You can test you regular expression using the regex101.com site
* Each student will have to write a short report of one to two pages and comment briefly the results. In your report, you must produce the tabulated results of your analysis as described below.

## Programming

### Imports

Some imports you may need. Add others as needed.

In [1]:
import bz2
import math
import os
import regex as re
import requests
import sys
from zipfile import ZipFile

### Collecting and analyzing a corpus

Retrieve a corpus of novels by Selma Lagerl&ouml;f from this URL:
<a href="https://github.com/pnugues/ilppp/blob/master/programs/corpus/Selma.txt">
    <tt>https://github.com/pnugues/ilppp/blob/master/programs/corpus/Selma.txt</tt>
</a>. The text of these novels was extracted
from <a href="https://litteraturbanken.se/forfattare/LagerlofS/titlar">Lagerlöf arkivet</a> at
<a href="https://litteraturbanken.se/">Litteraturbanken</a>.

In [2]:
# You may have to adjust the path
corpus = open('corpus/Selma.txt', encoding='utf8').read()

Run the <a href="https://github.com/pnugues/ilppp/tree/master/programs/ch02/python">concordance
program </a> to print the lines containing a specific word, for instance <i>Nils</i>.

In [3]:
pattern = 'Nils Holgersson'
width = 25

In [4]:
# spaces match tabs and newlines
pattern = re.sub(' ', r'\\s+', pattern)
# Replaces newlines with spaces in the text
clean_corpus = re.sub(r'\s+', ' ', corpus)
concordance = ('(.{{0,{width}}}{pattern}.{{0,{width}}})'
               .format(pattern=pattern, width=width))
for match in re.finditer(concordance, clean_corpus):
    print(match.group(1))
# print the string with 0..width characters on either side

Selma Lagerlöf Nils Holgerssons underbara resa genom Sv
! Se på Tummetott! Se på Nils Holgersson Tummetott!» Genast vände
r,» sade han. »Jag heter Nils Holgersson och är son till en husma
lden. »Inte är det värt, Nils Holgersson, att du är ängslig eller
 i dem. På den tiden, då Nils Holgersson drog omkring med vildgäs
ulle allt visa honom vad Nils Holgersson från Västra Vemmenhög va
om ägde rum det året, då Nils Holgersson for omkring med vildgäss
m vad det kan kosta dem. Nils Holgersson hade inte haft förstånd 
de det inte mer sägas om Nils Holgersson, att han inte tyckte om 
 Rosenbom?» För där stod Nils Holgersson mitt uppe på Rosenboms n
 Med ens fingo de syn på Nils Holgersson, och då sköt den store v
vila. När vildgässen och Nils Holgersson äntligen hade letat sig 
 slags arbetare. Men vad Nils Holgersson inte såg, det var, att s
nde han fråga, och om då Nils Holgersson sade nej, började han ge
de lille Mats, och om nu Nils Holgersson också hade tegat, så had
åg så försmädlig ut,

Run a simple <a href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">tokenization
program</a> on your corpus.

In [5]:
def tokenize(text):
    words = re.findall(r'\p{L}+', text)
    return words

In [6]:
words = tokenize(corpus)
words[:10]

['Selma',
 'Lagerlöf',
 'Nils',
 'Holgerssons',
 'underbara',
 'resa',
 'genom',
 'Sverige',
 'Första',
 'bandet']

Count the number of unique words in the original corpus and when setting all the words in lowercase

Original text

In [7]:
# Write your code here
print(len(set(words)))

44256


Lowercased text

In [8]:
# Write your code here
lower_case = [w.lower() for w in words]

### Segmenting a corpus

You will write a program to tokenize your text, insert `<s>` and `</s>` tags to delimit sentences, and set all the words in lowercase letters. In the end, you will only keep the words.

#### Normalizing 

Write a regular expression that matches all the characters that are neither a letter nor a punctuation sign. The punctuations signs will be the followings: `.;:?!`. In your regex, use the same order. For the definition of a letter, use a Unicode regex. You will call the regex string `nonletter`

In [9]:
# Write your code
nonletter = re.compile(r"[^\p{L}\.;:?!]")

Write a `clean()` function that replaces all the characters that are neither a letter nor a punctuation sign with a space. The punctuations signs will be the followings: `.;:?!`.   For the sentence:

_En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa._

the result will be:

`En gång hade de på Mårbacka en barnpiga som hette Back Kajsa.`

In [10]:
# Write your code here
def clean(text):
    cleaned_text = nonletter.sub(' ', text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    return cleaned_text.strip()


In [11]:
test_para = 'En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa. \
Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, \
hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, \
när hon kammade dem, och till humöret var hon dyster och sorgbunden.'

In [12]:
test_para = clean(test_para)
test_para

'En gång hade de på Mårbacka en barnpiga som hette Back Kajsa. Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden.'

In [13]:
print(corpus[:200])

Selma Lagerlöf


Nils Holgerssons underbara resa genom Sverige. Första bandet





Bokutgåva


Albert Bonniers förlag, Stockholm 1907.




DEN KRISTLIGA DAGVISAN.





Den signade dag, som vi nu här s


In [14]:
print(clean(corpus[:200]))

Selma Lagerlöf Nils Holgerssons underbara resa genom Sverige. Första bandet Bokutgåva Albert Bonniers förlag Stockholm . DEN KRISTLIGA DAGVISAN. Den signade dag som vi nu här s


#### Segmenter

In this section, you will write a sentence segmenter that will delimit each sentence with `</s>` and `<s>` symbols. For example the sentence:

_En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa._

will be bracketed as:

`<s> En gång hade de på Mårbacka en barnpiga som hette Back-Kajsa </s>`

As algorithm, you will use a simple heuristics to detect the sentence boundaries: A sentence starts with a capital letter and ends with a period-equivalent punctuation sign. You will write a regex to match these boundaries with a regular expression and you will insert `</s>\n<s>` symbols with a substitution function.

##### Detecting sentence boundaries

Write a regular expression that matches a punctuation, a sequence of spaces, and an uppercase letter. Call this regex string `sentence_boundaries`. In the regex, you will remember the value of the uppercase letter using a backreference. Use the Unicode regexes for the letters and the spaces. You may use `\s` though.

In [15]:
# Write your code here
sentence_boundaries = r"[.;:?!]\s+(\p{Lu})\1?"
text = "Hello! This is a test. Another sentence? Lol! XD"
matches = re.findall(sentence_boundaries, text)
print(matches)


['T', 'A', 'L', 'X']


##### Replacement markup

Write a string to replace the matched boundaries with the sentence boundary markup. Remember that a sentence ends with `</s>` and starts with `<s>` and that there is one sentence per line. Hint: The markup is `</s>\n<s>`. Remember also that the first letter of your sentence is in a regex backreference. Call the regex string `sentence_markup`.

In [16]:
# Write your code here
sentence_markup  = r" </s>\n<s> \1"
testing_sentence = re.sub(sentence_boundaries, sentence_markup, text)
print(testing_sentence)


Hello </s>
<s> This is a test </s>
<s> Another sentence </s>
<s> Lol </s>
<s> XD


##### Applying the substitution

Use your regexes to segment your text. Use the string `sentence_boundaries`, `sentence_markup`, and `test_para` as input and `text` as output.

In [17]:
# Write your code here
text = re.sub(sentence_boundaries, sentence_markup, test_para)

In [18]:
print(text)

En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden.


Insert markup codes in the beginning and end of the text

In [19]:
# Write your code here
text = "<s> " +text+ " </s>"

In [20]:
print(text)

<s> En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden. </s>


Replace the space duplicates with one space and remove the punctuation signs. For the spaces, use the Unicode regex. You may use `\s` though.

In [21]:
# Write your code here
s = r'\p{Zs}+'
p = r'[.;:?!]'
_t = re.sub(s, ' ', '\n  Hello', re.MULTILINE)
assert  _t == '\n Hello', _t
text = re.sub(p, '', text)
text = re.sub(r'\n+', '\n', text)
text = re.sub(s, ' ', text)

In [22]:
print(text)

<s> En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden </s>


Write a `segment_sentences(text)` function to gather the code in the Segmenter section and set the text in lowercase

In [23]:
# Write your code here
def segment_sentences(t):
    t = re.sub(sentence_boundaries, sentence_markup, t)
    _text = "<s> " +t+ " </s>"
    _text = re.sub(p, '', _text)
    _text = re.sub(r'\n+', '\n', _text)
    _text = re.sub(s, ' ', _text)
    _text = re.sub(r',', '', _text) ##WTF
    return _text.lower()

    

In [24]:
print(segment_sentences(test_para))

<s> en gång hade de på mårbacka en barnpiga som hette back kajsa </s>
<s> hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden </s>


Estimate qualitatively the accuracy of your program.

#### Tokenizing the corpus

Clean and segment the corpus using the functions you have written

In [71]:
# Write your code here
corpus = segment_sentences(corpus)


The result should be a normalized text without punctuation signs where all the sentences are delimited with `<s>` and `</s>` tags. The five last lines of the text should look like this. You may have some small differences.

In [72]:
print(corpus[-557:])

on hade fått större kärlek av sina föräldrar än någon annan han visste och sådan kärlek måste vändas i välsignelse </s>
<s> då prästen sade detta kom alla människor att se bort mot klara gulla och de förundrade sig över vad de såg </s>
<s> prästens ord tycktes redan ha gått i uppfyllelse </s>
<s> där stod klara fina gulleborg ifrån skrolycka hon som var uppkallad efter själva solen vid sina föräldrars grav och lyste som en förklarad </s>
<s> hon var likaså vacker som den söndagen då hon gick till kyrkan i den röda klänningen om inte vackrare </s> </s>


You will now create a list of words from your string. You will consider that a space or a carriage return is an item separator

In [73]:
# Write your code here
words = words = re.findall(r'\S+', corpus)

The five last lines of the corpus should like this:

In [74]:
print(words[-101:])

['hon', 'hade', 'fått', 'större', 'kärlek', 'av', 'sina', 'föräldrar', 'än', 'någon', 'annan', 'han', 'visste', 'och', 'sådan', 'kärlek', 'måste', 'vändas', 'i', 'välsignelse', '</s>', '<s>', 'då', 'prästen', 'sade', 'detta', 'kom', 'alla', 'människor', 'att', 'se', 'bort', 'mot', 'klara', 'gulla', 'och', 'de', 'förundrade', 'sig', 'över', 'vad', 'de', 'såg', '</s>', '<s>', 'prästens', 'ord', 'tycktes', 'redan', 'ha', 'gått', 'i', 'uppfyllelse', '</s>', '<s>', 'där', 'stod', 'klara', 'fina', 'gulleborg', 'ifrån', 'skrolycka', 'hon', 'som', 'var', 'uppkallad', 'efter', 'själva', 'solen', 'vid', 'sina', 'föräldrars', 'grav', 'och', 'lyste', 'som', 'en', 'förklarad', '</s>', '<s>', 'hon', 'var', 'likaså', 'vacker', 'som', 'den', 'söndagen', 'då', 'hon', 'gick', 'till', 'kyrkan', 'i', 'den', 'röda', 'klänningen', 'om', 'inte', 'vackrare', '</s>', '</s>']


### Counting unigrams and bigrams

Read and try programs to compute the frequency of unigrams and bigrams of the training set in the noteboo: [https://github.com/pnugues/pnlp/blob/main/notebooks/10_01_ngrams.ipynb](https://github.com/pnugues/pnlp/blob/main/notebooks/10_01_ngrams.ipynb).

#### Unigrams

In [75]:
def unigrams(words):
    frequency = {}
    for i in range(len(words)):
        if words[i] in frequency:
            frequency[words[i]] += 1
        else:
            frequency[words[i]] = 1
    return frequency

We compute the frequencies.

In [76]:
frequency = unigrams(words)
list(frequency.items())[:20]

[('<s>', 45040),
 ('selma', 48),
 ('lagerlöf', 267),
 ('nils', 86),
 ('holgerssons', 6),
 ('underbara', 23),
 ('resa', 301),
 ('genom', 688),
 ('sverige', 52),
 ('</s>', 45040),
 ('första', 519),
 ('bandet', 6),
 ('bokutgåva', 11),
 ('albert', 15),
 ('bonniers', 11),
 ('förlag', 11),
 ('stockholm', 76),
 ('1907', 2),
 ('den', 11435),
 ('kristliga', 2)]

#### Bigrams

In [77]:
def bigrams(words):
    bigrams = []
    for i in range(len(words) - 1):
        bigrams.append((words[i], words[i + 1]))
    frequency_bigrams = {}
    for i in range(len(words) - 1):
        if bigrams[i] in frequency_bigrams:
            frequency_bigrams[bigrams[i]] += 1
        else:
            frequency_bigrams[bigrams[i]] = 1
    return frequency_bigrams

In [32]:
frequency_bigrams = bigrams(words)
list(frequency_bigrams.items())[:20]

[(('<s>', 'selma'), 5),
 (('selma', 'lagerlöf'), 10),
 (('lagerlöf', 'nils'), 1),
 (('nils', 'holgerssons'), 6),
 (('holgerssons', 'underbara'), 4),
 (('underbara', 'resa'), 4),
 (('resa', 'genom'), 6),
 (('genom', 'sverige'), 5),
 (('sverige', '</s>'), 14),
 (('</s>', '<s>'), 45038),
 (('<s>', 'första'), 9),
 (('första', 'bandet'), 1),
 (('bandet', 'bokutgåva'), 2),
 (('bokutgåva', 'albert'), 11),
 (('albert', 'bonniers'), 11),
 (('bonniers', 'förlag'), 11),
 (('förlag', 'stockholm'), 10),
 (('stockholm', '1907'), 2),
 (('1907', '</s>'), 2),
 (('<s>', 'den'), 1160)]

In the report, tell what is the possible number of bigrams and their real number? Explain why such a difference. What would be the possible number of 4-grams.

Propose a solution to cope with bigrams unseen in the corpus.

### Computing the likelihood of a sentence
You will now compute the likelihood of a sentence using a bigram and a trigram models. 

In both models, you will ignore the start of sentence symbol, `<s>`, as this factor is common to both models: $P(<s>)$. This will save you one multiplication.

#### Unigrams

Write a program to compute a sentence's probability using unigrams. Your function will return the perplexity.

Your function should print and tabulate the results as in the examples below with the sentence _Det var en gång en katt som hette Nils_.

Your figures might be slightly different because of differences in the sentence segmentation.

```
=====================================================
wi 	 C(wi) 	 #words 	 P(wi)
=====================================================
det 	 21108 	 1041631 	 0.0202643738521607
var 	 12090 	 1041631 	 0.01160679741674355
en 	 13514 	 1041631 	 0.01297388422579589
gång 	 1332 	 1041631 	 0.001278763784871994
en 	 13514 	 1041631 	 0.01297388422579589
katt 	 16 	 1041631 	 1.5360525944408337e-05
som 	 16288 	 1041631 	 0.015637015411407686
hette 	 97 	 1041631 	 9.312318853797554e-05
nils 	 87 	 1041631 	 8.352285982272032e-05
</s> 	 59047 	 1041631 	 0.056687060964967444
=====================================================
Prob. unigrams:	 5.361459667285409e-27
Geometric mean prob.: 0.0023600885848765307
Entropy rate:	 8.726943273141258
Perplexity:	 423.71290908655254
```

In [78]:
# Write your code
length = len(words)
def unigram_lm(frequency, sent_words):
    """
    Computing the sentence probs with a unigram model.
    """
    data = []
    prob_unigram = 1
    for word in sent_words:
        prob = frequency.get(word, 0)/length
        prob_unigram *= prob
        data.append([word, frequency.get(word, 0), length, prob])

    geo_mean = prob_unigram ** (1/len(sent_words))
    entropy = -math.log2(geo_mean)
    perplexity = 2 ** entropy
    print("=" * 65)
    print(f"{'wi':<10}{'C(wi)':>10}{'#words':>12}{'P(wi)':>20}")
    print("=" * 65)
    for row in data:
        print(f"{row[0]:<10}{row[1]:>10}{row[2]:>12}{row[3]:>20.10f}")
    print("=" * 65)

    print(f"{'Prob. unigrams:':<25}{prob_unigram:.10e}")
    print(f"{'Geometric mean prob.:':<25}{geo_mean:.10f}")
    print(f"{'Entropy rate:':<25}{entropy:.6f}")
    print(f"{'Perplexity:':<25}{perplexity:.6f}")
    print("=" * 65)



    return perplexity

    

In [79]:
sentence = 'kajsa hade en nils som var stor och stark </s>'
sent_words = sentence.split()
sent_words

['kajsa', 'hade', 'en', 'nils', 'som', 'var', 'stor', 'och', 'stark', '</s>']

In [80]:
perplexity_unigrams = unigram_lm(frequency, sent_words)

wi             C(wi)      #words               P(wi)
kajsa             42     1018050        0.0000412553
hade           13189     1018050        0.0129551594
en             13441     1018050        0.0132026914
nils              86     1018050        0.0000844752
som            16275     1018050        0.0159864447
var            11998     1018050        0.0117852758
stor             948     1018050        0.0009311920
och            36090     1018050        0.0354501252
stark            277     1018050        0.0002720888
</s>           45040     1018050        0.0442414420
Prob. unigrams:          4.4627579639e-26
Geometric mean prob.:    0.0029171606
Entropy rate:            8.421219
Perplexity:              342.799095


In [68]:
perplexity_unigrams = int(perplexity_unigrams)
perplexity_unigrams

342

#### Bigrams

Write a program to compute the sentence probability using bigrams. Your function will tabulate and print the results as below. It will return the perplexity.

```
=====================================================
wi 	 wi+1 	 Ci,i+1 	 C(i) 	 P(wi+1|wi)
=====================================================
<s>	 det 	 5672 	 59047 	 0.09605907158704083
det 	 var 	 3839 	 21108 	 0.1818741709304529
var 	 en 	 712 	 12090 	 0.058891645988420185
en 	 gång 	 706 	 13514 	 0.052242119283705785
gång 	 en 	 20 	 1332 	 0.015015015015015015
en 	 katt 	 6 	 13514 	 0.0004439840165754033
katt 	 som 	 2 	 16 	 0.125
som 	 hette 	 45 	 16288 	 0.002762770137524558
hette 	 nils 	 0 	 97 	 0.0 	 *backoff: 	 8.352285982272032e-05
nils 	 </s> 	 2 	 87 	 0.022988505747126436
=====================================================
Prob. bigrams:	 2.376007803503683e-19
Geometric mean prob.: 0.013727289294133601
Entropy rate:	 6.186809422848149
Perplexity:	 72.84759420254609
```

In [69]:
# Write your code
def bigram_lm(frequency, frequency_bigrams, sent_words):
    data = []
    _bigrams = [("<s>", sent_words[0])]
    for i in range(0, len(sent_words) - 1):
        _bigrams.append((sent_words[i], sent_words[i + 1]))
    bigram_prob = 1
    for bigram in _bigrams:
        c_i_i1 = frequency_bigrams.get(bigram, 0)
        c_i = frequency.get(bigram[0], 0)
        p_wi1_wi = (c_i_i1 / c_i if c_i > 0 else 0) if c_i_i1 > 0 else frequency.get(bigram[1])/length
        bigram_prob *= p_wi1_wi
        data.append([bigram[0], bigram[1], c_i_i1, c_i, p_wi1_wi])
    geo_mean = bigram_prob ** (1/len(_bigrams))
    entropy = -math.log2(geo_mean) if geo_mean > 0 else float('inf')
    perplexity = 2 ** entropy
    print("=" * 65)
    print(f"{'wi':<10}{'wi+1':>10}{'Ci,i+1':>12}{'C(i)':>12}{'P(wi+1|wi)':>20}")
    print("=" * 65)
    for row in data:
        print(f"{row[0]:<10}{row[1]:>10}{row[2]:>12}{row[3]:>12}{row[4]:>20.10f}")
    print("=" * 65)

    print(f"{'Prob. bigrams:':<25}{bigram_prob:.10e}")
    print(f"{'Geometric mean prob.:':<25}{geo_mean:.10f}")
    print(f"{'Entropy rate:':<25}{entropy:.6f}")
    print(f"{'Perplexity:':<25}{perplexity:.6f}")
    print("=" * 65)

    return perplexity

In [70]:
perplexity_bigrams = bigram_lm(frequency, frequency_bigrams, sent_words)

wi              wi+1      Ci,i+1        C(i)          P(wi+1|wi)
<s>            kajsa           1       45039        0.0000222030
kajsa           hade           3          42        0.0714285714
hade              en         206       13189        0.0156190765
en              nils           0       13441        0.0000844754
nils             som           0          86        0.0159864761
som              var         590       16275        0.0362519201
var             stor          22       11998        0.0018336389
stor             och          65         948        0.0685654008
och            stark          23       36090        0.0006372956
stark           </s>           8         277        0.0288808664
Prob. bigrams:           2.8062253551e-24
Geometric mean prob.:    0.0044137960
Entropy rate:            7.823764
Perplexity:              226.562353


In [136]:
perplexity_bigrams = int(perplexity_bigrams)
perplexity_bigrams

77

In addition to this sentence, _Det var en gång en katt som hette Nils_, write two other sentences that will form your test set and run your programs on them. You will insert them in your report.

### Online prediction of words

You will now carry out an online prediction of words. You will consider two cases:
1. Prediction of the current word a user is typing;
2. Prediction of the next word.

Ideally, you would write a loop that reads the words and apply the models while typing. As the Jupyter labs are not designed for interactive input and output, we will simplify the experimental settings with constant strings at a given time of the input.  

We will assume the user is typing the phrase: _Det var en gång_. 

#### Trigrams

To have a more accurate prediction, you will use a trigram counting function. Program this function following the model of your bigram counting function.

In [137]:
# Write your code
def trigrams(words):
    trigrams = []

    for i in range(len(words) - 2):
        trigrams.append((words[i], words[i + 1], words[i + 2]))

    frequency_trigrams = {}
    for trigram in trigrams:
        if trigram in frequency_trigrams:
            frequency_trigrams[trigram] += 1
        else:
            frequency_trigrams[trigram] = 1

    return frequency_trigrams

    

In [138]:
frequency_trigrams = trigrams(words)
frequency_trigrams[('det', 'var', 'en')]

307

#### Prediction

The user starts typing _Det var en gång_. After the 2nd character, your program tries to help the user with suggested words.

In [139]:
starting_text = 'De'.lower()
starting_text

'de'

Write a program to rank the five first candidates at this point. Assign these predictions in a list that you will call `current_word_predictions_1`. Note that you are starting a sentence and you can then use the bigram frequencies. Write a sorting key that will enable you to have a deterministic ranking of the words or bigrams with identical frequencies: When two words have the same frequency, you will sort them by alphabetic order. You can do this with a tuple.

In [140]:
cand_nbr = 5

In [141]:
# Write your code here

candidates = {}

for (w1,w2), freq in frequency_bigrams.items():
    if w1 == '<s>' and w2.startswith('de'):
        candidates[w2] = freq

sorted_candidates = sorted(
    candidates.items(),
    key=lambda item: (-item[1], item[0].lower())
)

current_word_predictions_1 = [word for word, freq in sorted_candidates[:cand_nbr]]



In [142]:
current_word_predictions_1

['det', 'de', 'den', 'detta', 'denna']

Let us now suppose that the user has typed: _Det var en_. After detecting a space, your program starts predicting a next possible word.

In [143]:
current_text = "Det var en ".lower()
current_text

'det var en '

Tokenize this text and return a list of tokens. Call it `tokens`.

In [144]:
# Write your code here
tokens = current_text.lower().strip().split(" ")

In [145]:
tokens

['det', 'var', 'en']

Write a program to propose the five next possible words ranked by frequency using a trigram model. Assign these predictions to a variable that you will call `next_word_predictions`. Write a sorting key that will enable you have a deterministic ranking of the words or bigrams with identical frequencies: When two words have the same frequency, you will sort them by alphabetic order. You can do this with a tuple.

In [146]:
# Write your code here
full_text = tokens.copy()
best_five_words = []

full_text = tokens.copy()
best_five_words = []

def findBestNextCandidate(word1, word2):
    candidates = {}
    for (w1, w2, w3), freq in frequency_trigrams.items():
        if w1 == word1 and w2 == word2:
            candidates[w3] = freq

    sorted_candidates = sorted(
        candidates.items(),
        key=lambda item: (-item[1], item[0].lower())
    )
    return [word for word, freq in sorted_candidates[:cand_nbr]]



In [147]:
next_word_predictions = findBestNextCandidate(tokens[1], tokens[2])
print(next_word_predictions)

['stor', 'liten', 'gammal', 'god', 'sådan']


Finally, let us suppose that the user has typed _Det var en g_, rank the five possible candidates. Assign these predictions in a list that you will call `current_word_predictions_2`

In [148]:
current_text = "Det var en g".lower()

In [149]:
# Write your code here
def findBestAutocompletion(token):
    word1 = token[1]
    word2 = token[2]
    incomplete_word = token[3]

    candidates = {}
    for (w1, w2, w3), freq in frequency_trigrams.items():
        if w1 == word1 and w2 == word2 and w3.startswith(incomplete_word):
            candidates[w3] = freq

    sorted_candidates = sorted(
        candidates.items(),
        key=lambda item: (-item[1], item[0].lower())
    )
    return [word for word, freq in sorted_candidates[:cand_nbr]]

In [150]:
current_word_predictions_2 = findBestAutocompletion(tokenize(current_text))
print(current_word_predictions_2)

['gammal', 'god', 'gång', 'ganska', 'glädje']


## Turning in your assignment

Now your are done with the program. To complete this assignment, you will:
1. Write a short individual report on your program. I suggest that you use this structure for your report:
      1. Objectives and dataset
      2. Method and program structure, where you should outline your program and possibly describe difficult parts. In this section, you will include the __regular expression you used to segment the text__.
      3. Results. You will include the __unigram and bigram tables__ for _Det var en gång en katt som hette Nils_ and __two other sentences__. You will also include the results of your __next word prediction__.
      4. Conclusion
      5. Answer to possible questions
2. Execute the Jupyter notebook by Peter Norvig here: <a href="http://nbviewer.jupyter.org/url/norvig.com/ipython/How%20to%20Do%20Things%20with%20Words.ipynb">https://nbviewer.jupyter.org/url/norvig.com/ipython/How to Do Things with Words.ipynb</a>. Just run all the cells and be sure that you understand the code. You will find the data here: <a href="http://norvig.com/ngrams/">http://norvig.com/ngrams/</a>.
3. In your report, after the description of your program and conclusion, you will describe one experiment with Norvig's notebook and a __long string of words your will create yourself or copy from a text you like__. You will remove all the punctuation and white spaces from this string. You will set this string in lowercase letters. You will just add a cell at the end of Sect. 7 in Norvig's notebook, where you will use your string and run the notebook cell with the <tt>segment()</tt> and <tt>segment2()</tt> functions. __You will comment the segmentation results you obtained__ with the unigram and bigram models.

Submit your report as well as your **notebook** (for archiving purposes) to Canvas: https://canvas.education.lu.se/. To write your report, use Latex. This will probably help you structure your text. You can use the Overleaf online editor (www.overleaf.com). You will then upload a PDF file in Canvas.

The submission deadline is September 26, 2025. You will have only three submission attempts. The deadline for the second and third ones are one week after you are noticed of your result.